In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Import Data

In [3]:
# Import solar data
# https://www.californiadgstats.ca.gov/downloads/
# CSI Working Data Set
raw = pd.read_csv('WorkingDataSet_5-28-2020.csv')

C:\Users\mcarney\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (27,29,42,43,51,52,53,54,55,56,57,60,61,62,63,64,65,66,80,81,82,83,84,90,91,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
raw.head(1)

,Application Number,Program Administrator,Program,Incentive Design,Incentive Type,Incentive Step,Incentive Amount,Total Cost,Nameplate Rating,CEC PTC Rating,...,CEC PTC Rating Single-Axis Tracking,CEC PTC Rating Dual-Axis Tracking,CSI Rating Fixed,CSI Rating Single-Axis Tracking,CSI Rating Dual-Axis Tracking,MASH Track 1A Incentive Amount,MASH Track 1B Incentive Amount,MASH Track 2 Incentive Amount,MASH Track 1A % Capacity,MASH Track 1B % Capacity
0,SD-CSI-03095,CSE,Small Commercial (< 10 kW) and All Residential,$1.55 per Watt EPBB,EPBB,Step 5,3446.0,0.0,2.816,2.286,...,0.0,0.0,2.223,0.0,0.0,NaN,NaN,NaN,NaN,NaN
